In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# STEP 3: Load and Clean Data
def load_data(path='fra.txt', num_examples = 50000):
    with open(path, encoding='utf-8') as f:
        lines = f.read().strip().split('\n')
    
    pairs = [line.split('\t')[:2] for line in lines[:num_examples]]
    
    input_texts = []
    target_texts = []
    for eng, fra in pairs:
        input_texts.append(eng)
        target_texts.append(f"<start> {fra} <end>")
    
    return input_texts, target_texts

input_texts, target_texts = load_data()

In [3]:
# STEP 4: Tokenize
input_tokenizer = Tokenizer(filters='')
target_tokenizer = Tokenizer(filters='')

input_tokenizer.fit_on_texts(input_texts)
target_tokenizer.fit_on_texts(target_texts)

input_seqs = input_tokenizer.texts_to_sequences(input_texts)
target_seqs = target_tokenizer.texts_to_sequences(target_texts)


In [4]:
# STEP 5: Pad sequences
input_tensor = pad_sequences(input_seqs, padding='post')
target_tensor = pad_sequences(target_seqs, padding='post')

print("Input tensor shape:", input_tensor.shape)
print("Target tensor shape:", target_tensor.shape)


Input tensor shape: (50000, 7)
Target tensor shape: (50000, 14)


In [5]:
# STEP 7: Vocabulary Sizes
input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

print("Input vocab size:", input_vocab_size)
print("Target vocab size:", target_vocab_size)


Input vocab size: 9130
Target vocab size: 17458


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense


In [7]:
embedding_dim = 256
units = 512

input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1


In [8]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# Save encoder states to initialize decoder
encoder_states = [state_h, state_c]


In [9]:
# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(target_vocab_size, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

decoder_dense = Dense(target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [10]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    2337280     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    4469248     ['input_2[0][0]']                
                                                                                              

In [11]:
# Remove the first <start> token from target and prepare as labels
decoder_target_data = target_tensor[:, 1:]


In [12]:
batch_size = 64
epochs = 10

model.fit(
    [input_tensor, target_tensor[:, :-1]],
    tf.expand_dims(decoder_target_data, -1),
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)


Epoch 1/10
625/625 [==============================] - 24s 33ms/step - loss: 2.0670 - val_loss: 2.1116
Epoch 2/10
625/625 [==============================] - 21s 33ms/step - loss: 1.4513 - val_loss: 1.8227
Epoch 3/10
625/625 [==============================] - 20s 32ms/step - loss: 1.1630 - val_loss: 1.6864
Epoch 4/10
625/625 [==============================] - 20s 32ms/step - loss: 0.9703 - val_loss: 1.6299
Epoch 5/10
625/625 [==============================] - 20s 32ms/step - loss: 0.8210 - val_loss: 1.5959
Epoch 6/10
625/625 [==============================] - 20s 32ms/step - loss: 0.6986 - val_loss: 1.5701
Epoch 7/10
625/625 [==============================] - 20s 32ms/step - loss: 0.5953 - val_loss: 1.5694
Epoch 8/10
625/625 [==============================] - 20s 32ms/step - loss: 0.5061 - val_loss: 1.5693
Epoch 9/10
625/625 [==============================] - 20s 32ms/step - loss: 0.4300 - val_loss: 1.5633
Epoch 10/10
625/625 [==============================] - 20s 32ms/step - loss: 0.365

In [13]:
# Encoder model for inference (takes input sentence, returns hidden states)
encoder_model = Model(encoder_inputs, encoder_states)


In [14]:
# Decoder inputs
decoder_state_input_h = Input(shape=(units,))
decoder_state_input_c = Input(shape=(units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Decoder embedding
dec_emb_inf = dec_emb_layer(decoder_inputs)

# LSTM with previous states
decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(
    dec_emb_inf, initial_state=decoder_states_inputs)

decoder_states_inf = [state_h_inf, state_c_inf]
decoder_outputs_inf = decoder_dense(decoder_outputs_inf)

# Full decoder inference model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs_inf] + decoder_states_inf)

In [15]:
target_idx_word = {i: w for w, i in target_tokenizer.word_index.items()}

def decode_sequence(input_seq):
    # Encode the input
    states_value = encoder_model.predict(input_seq)

    # Start with <start> token
    target_seq = np.array([[target_tokenizer.word_index['<start>']]])

    decoded_sentence = ''
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample the token with highest probability (greedy search)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = target_idx_word.get(sampled_token_index, '')

        if sampled_word == '<end>' or len(decoded_sentence.split()) > 50:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word

        # Update the target sequence and states
        target_seq = np.array([[sampled_token_index]])
        states_value = [h, c]

    return decoded_sentence.strip()


In [16]:
# Pick 5 random sentences to test
for i in range(5):
    input_seq = input_tensor[i:i+1]
    decoded = decode_sequence(input_seq)
    
    print(f"Input    : {input_texts[i]}")
    print(f"Predicted: {decoded}")
    print(f"Target   : {target_texts[i].replace('<start>', '').replace('<end>', '').strip()}")
    print("-" * 50)


1/1 [==============================] - 0s 18ms/step
Input    : Go.
Predicted: en route !
Target   : Va !
--------------------------------------------------
1/1 [==============================] - 0s 18ms/step
Input    : Go.
Predicted: en route !
Target   : Marche.
--------------------------------------------------
1/1 [==============================] - 0s 18ms/step
Input    : Go.
Predicted: en route !
Target   : En route !
--------------------------------------------------
1/1 [==============================] - 0s 18ms/step
Input    : Go.
Predicted: en route !
Target   : Bouge !
--------------------------------------------------
1/1 [==============================] - 0s 18ms/step
Input    : Hi.
Predicted: salut !
Target   : Salut !
--------------------------------------------------


In [17]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class BahdanauAttention(Layer):
    def __init__(self, units):
        super().__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(query_with_time_axis)))

        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, tf.squeeze(attention_weights, -1)


In [18]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(dec_units, return_sequences=True, return_state=True)
        self.attention = BahdanauAttention(dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state_h, state_c = self.lstm(x)
        x = self.fc(output)
        return x, state_h, state_c, attention_weights


In [19]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_attention(attention, input_sentence, predicted_sentence):
    fig = plt.figure(figsize=(10, 8))
    sns.heatmap(attention, xticklabels=input_sentence.split(), yticklabels=predicted_sentence.split(), cmap='viridis')
    plt.xlabel('Input')
    plt.ylabel('Predicted')
    plt.title("Attention Heatmap")
    plt.show()

In [20]:
import matplotlib.pyplot as plt

def plot_training_curves(history):
    plt.figure(figsize=(12, 5))

    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Accuracy (optional)
    if 'accuracy' in history.history:
        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Train Acc')
        if 'val_accuracy' in history.history:
            plt.plot(history.history['val_accuracy'], label='Val Acc')
        plt.title('Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

    plt.tight_layout()
    plt.show()
